In [1]:
import pandas as pd
import gtfstk as gt
import geopandas as gpd
import tkinter as tk
import requests, zipfile, io, os, re, shutil
from tkinter import filedialog
from shapely.geometry import Point
%matplotlib inline

In [2]:
idaho='16'
ada_county='001'
canyon_county='027'
places_url=r"https://www2.census.gov/geo/tiger/TIGER2018/PLACE/tl_2018_16_place.zip"
counties_url=r"https://www2.census.gov/geo/tiger/TIGER2018/COUNTY/tl_2018_us_county.zip"
crs={'init':'epsg:4326'}

In [3]:
def gtfs_clicked():
    root.gtfs=filedialog.askopenfilename(initialdir="/",
                                            title="Select GTFS Zip File",
                                            filetypes=(("zip files","*.zip"),("all files","*.*")))
def save_clicked():
    root.save_file=filedialog.asksaveasfilename(initialdir="/",
                                                  title="Save",
                                                  defaultextension=".xlsx",
                                                  filetypes = (("xlsx files","*.xlsx"),("all files","*.*")))
def close_window():
    root.destroy()

root=tk.Tk()
root.title("Stops By Jurisdiction")
    
tk.Label(root,text="Select GTFS Zip File").grid(column=0,row=0)
tk.Button(root,text="Browse",command=gtfs_clicked).grid(column=1,row=0)

tk.Label(root,text="Save").grid(column=0,row=1)
tk.Button(root,text="Browse",command=save_clicked).grid(column=1,row=1)

tk.Button(root,text="Run",width=25,command=close_window).grid(column=0,row=2)
root.mainloop()

In [4]:
def readOnlineShapefile(url):
    r=requests.get(url)
    z=zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('x')
    gdf=gpd.read_file('x')
    shutil.rmtree('x')
    return gdf
places=readOnlineShapefile(places_url)

In [5]:
cities=['Boise City',
        'Caldwell',
        'Eagle',
        'Garden City',
        'Greenleaf',
        'Kuna',
        'Melba',
        'Meridian',
        'Middleton',
        'Nampa',
        'Notus',
        'Parma',
        'Star',
        'Wilder']
places=places.loc[places['NAME'].isin(cities)]
places.sort_values('NAME',inplace=True)

In [6]:
def getVRTServiceArea(url):
    gdf=readOnlineShapefile(url)
    gdf=gdf.loc[(gdf['STATEFP']==idaho)&((gdf['COUNTYFP']==ada_county)|(gdf['COUNTYFP']==canyon_county))]
    return gdf
counties=getVRTServiceArea(counties_url)

In [7]:
unincorporated=gpd.overlay(counties,places,how='difference')
jurisdictions=gpd.GeoDataFrame(pd.concat([places,unincorporated],ignore_index=True),crs=crs)

c:\users\bparker\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [8]:
jurisdictions['NAMELSAD'].replace({'city':''},regex=True,inplace=True)
jurisdictions['NAMELSAD']=jurisdictions['NAMELSAD'].str.strip()

cityNamesArray=jurisdictions.NAMELSAD.unique()
cityNames={i:cityNamesArray[i] for i in range(len(jurisdictions.NAMELSAD.unique()))}
cityNames[len(cityNames)+1]='All'
cityNames=cityNames.values()

In [12]:
def stopsByJurisdiction(path,save_file=None,city=None):
    feed=gt.read_gtfs(path,dist_units='mi')
    df=feed.stop_times.merge(gt.stops.geometrize_stops(feed.stops)).merge(feed.trips).merge(feed.routes)
    df=df.groupby(['stop_id','route_short_name'],as_index=False).first()
    gdf=gpd.GeoDataFrame(df,geometry='geometry',crs=crs)
    gdf=gpd.sjoin(gdf,jurisdictions,how='left')
    if city:
        gdf=gdf.loc[gdf['NAMELSAD']==city]
    cities=gdf.groupby(['route_short_name','NAMELSAD'],as_index=False)['stop_id'].count()
    routes=gdf.groupby('route_short_name',as_index=False)['stop_id'].count()
    routes.head()
    df=cities.merge(routes,on='route_short_name',how='left',suffixes=('_city','_total'))
    df.sort_values('route_short_name',inplace=True)
    df['Percent']=df['stop_id_city']/df['stop_id_total']
    df.rename({'route_short_name':'Route','NAMELSAD':'City','stop_id_city':'Stops in City','stop_id_total':'Total Stops'},axis=1,inplace=True)
    if save_file:
        df.to_excel(save_file,index=False)
    return df
stopsByJurisdiction(root.gtfs,root.save_file)

,Route,City,Stops in City,Total Stops,Percent
0,102c,Boise City,64,64,1.000000
1,103c,Boise City,43,51,0.843137
2,103c,Meridian,8,51,0.156863
3,106c,Boise City,60,72,0.833333
4,106c,Garden City,9,72,0.125000
5,106c,Meridian,3,72,0.041667
6,201a,Ada County,2,41,0.048780
7,201a,Boise City,13,41,0.317073
8,201a,Kuna,9,41,0.219512
9,201a,Meridian,17,41,0.414634
